# Finetune a Phi 3.5 model with ~ 4 billion parameters using LoRA

We use a self-curated dataset with [arXiv](https://arxiv.org/) as a basis. The whole dataset is available from [Kaggle](https://www.kaggle.com/datasets/Cornell-University/arxiv). The dataset was filtered to have either `LLM` (only as a whole word), `Large Language Model`, `GRPO` or `GSPO` in the title of the abstract. 

We use the following system prompt:
```
You are an educated researcher and always answer in correct scientific terms.
You are very deep into LLMs and its methodologies. You are very creative.
```

The model should learn how to write abstracts about a certain topic. An example user prompt would be:
```
Write an abstract with the title 'Few-shot training LLMs for project-specific code-summarization'
```

The example abstract is taken from arXiv, so the answer will be:
```
Very large language models (LLMs), such as GPT-3 and Codex have achieved
state-of-the-art performance on several natural-language tasks, and show great
promise also for code. A particularly exciting aspect of LLMs is their knack
for few-shot and zero-shot learning: they can learn to perform a task with very
few examples. Few-shotting has particular synergies in software engineering,
where there are a lot of phenomena (identifier names, APIs, terminology, coding
patterns) that are known to be highly project-specific. However,
project-specific data can be quite limited, especially early in the history of
a project; thus the few-shot learning capacity of LLMs might be very relevant.
In this paper, we investigate the use few-shot training with the very large GPT
(Generative Pre-trained Transformer) Codex model, and find evidence suggesting
that one can significantly surpass state-of-the-art models for
code-summarization, leveraging project-specific training.
```

If finetuning works, we will observe that the models know about `GRPO` which
was introduced *after* the knowledge cutoff. Moreover, the models should adhere
to the format and scientific language of the abstract.

Implementation ideas and parts of the script from https://www.philschmid.de/fine-tune-llms-in-2025Notebook adapted from unsloth: https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Phi_3.5_Mini-Conversational.ipynb

In [ ]:
import torch
torch.cuda.get_device_name(0)

In [ ]:
model_name = "microsoft/Phi-3.5-mini-instruct"
dataset_name = 'llm-abstract-dataset.jsonl.xz'
output_dir = "runs/" + model_name.split("/")[-1] + "-" + dataset_name.split(".")[0]

In [ ]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = 2048,
    dtype = None, # autodetect
    load_in_4bit = False
)

In [ ]:
# prepare a peft model for fine-tuning with a lora adapter
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 42,
    use_rslora = False,  # rank stabilized LoRA
    loftq_config = None, # and LoftQ
)

Remember the template format:
```<|system|>
You are a helpful travel assistant.<|end|>
<|user|>
I am going to Paris, what should I see?<|end|>
<|assistant|>
Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.These are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world."<|end|>
```

This format is called `phi-3`. However, our dataset uses the `ShareGPT` style, so we have to convert.

In [ ]:
print(tokenizer.chat_template)

In [ ]:
def formatting_prompts_func(examples):
    convos = examples["messages"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

In [ ]:
from datasets import load_dataset
dataset = load_dataset("json", data_files=dataset_name, split="train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
dataset[1]["messages"]

In [ ]:
print(dataset[1]["text"])

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = True, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        #max_steps = 600,
        learning_rate = 2e-4,
        fp16 = False,
        bf16 = True,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

In [ ]:
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)
